# Analisis de datos

## 1 - ETL ( Extración, Transformación y Carga)

### 1. Extración (Extract)

In [12]:
from google.colab import drive
import pandas as pd
import chardet

# Montar google drive para acceder al archivo
drive.mount('/content/drive' , force_remount= True)

#ruta del csv
ruta_archivo = '/content/drive/MyDrive/AnalisisDeDatos/ventas.csv'
with open(ruta_archivo, 'rb') as f:
    result = chardet.detect(f.read())

#manejo de errores
try:
  df = pd.read_csv(ruta_archivo, encoding= result['encoding'], sep=';', decimal= ',')
  print('Extracion de datos es correcta')
except FileNotFoundError:
  print(f'Error al cargar el archivo en la ruta: {ruta_archivo} ')

Mounted at /content/drive
Extracion de datos es correcta


### 2. Transformación (Transform)

In [16]:
#Mostrar filas del DataFrame
df.head(10)

,tienda,marca,tipo,genero,talla,color,categoria,precio,fecha_hora
0,Lima,Asics,WB1820,F,42,Azul,Pantalon,89.00,2015-07-15 07:32:00
1,Lima,Asics,Kayano Single Tab,U,42-44,Azul,Ropa interior,24.99,2015-07-15 07:33:00
2,Lima,Asics,WB1820,F,37,Rosado,Pantalon,89.00,2015-07-15 07:52:00
3,Lima,Asics,WB2585,F,39,Negro,Pantalon,99.00,2015-07-15 07:58:00
4,Lima,Asics,WB1820,F,46,Multicolor,Pantalon,89.00,2015-07-15 08:19:00
5,Lima,Asics,WB1820,F,39,Azul,Pantalon,89.00,2015-07-15 08:48:00
6,Lima,Asics,MB1878,M,38,Negro,Pantalon,99.00,2015-07-15 09:07:00
7,Lima,Asics,Hera Deux Single Tab,F,42-44,Negro,Ropa interior,12.99,2015-07-15 09:11:00
8,Lima,Asics,MB1878,M,38,Negro,Pantalon,99.00,2015-07-15 10:04:00
9,Lima,Asics,MB1878,M,46,Negro,Pantalon,99.00,2015-07-15 10:06:00


In [ ]:
#Resumen de la inforacion del DataFrame
df.info()

#Valores talles
print(df.groupby('categoria')['talla'].unique())

#valores tipo
print(df.groupby('tipo')['talla'].unique())

#test
print(df.groupby(['tienda','marca'])['precio'].sum().reset_index())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   tienda      800 non-null    object        
 1   marca       800 non-null    object        
 2   tipo        800 non-null    object        
 3   genero      800 non-null    object        
 4   talla       800 non-null    object        
 5   color       800 non-null    object        
 6   categoria   800 non-null    object        
 7   precio      800 non-null    float64       
 8   fecha_hora  800 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 56.4+ KB


In [13]:
#LIMPIEZA DE DATOS
#renombrar columnas para facilitar acceso y evitar caracteres especiales
df.rename(columns = {
    'Tienda': 'tienda',
    'Marca': 'marca',
    'Tipo': 'tipo',
    'Gï¿½nero': 'genero',
    'Talla' : 'talla',
    'Color' : 'color',
    'Categoria': 'categoria',
    'precio de venta': 'precio',
    'Fecha': 'fecha',
    'Hora': 'hora'
}, inplace = True)

In [14]:

#transformar a variable numerica la columba precio
df['precio'] = df['precio'].astype(str)
df['precio'] = df['precio'].str.replace(',', '.', regex=True).astype(float)


#renombrar valores de columna genero
df['genero'] = df['genero'].str.replace('Masculino', 'M', regex=False)
df['genero'] = df['genero'].str.replace('Femenino', 'F', regex=False)
df['genero'] = df['genero'].str.replace('Usinex', 'U', regex=False)
print(set(df['genero']))

#crear nueva columna combinacion entre fecha y hora
df['fecha_hora'] = pd.to_datetime(df['fecha'] + ' ' + df['hora'], format='%d/%m/%Y %H:%M:%S')


#eliminar columna fecha y hora
df.drop(columns=['fecha', 'hora'], inplace=True)


{'F', 'M', 'U'}


In [15]:
#estadistica descriptiva
df.describe()

,precio,fecha_hora
count,800.000000,800
mean,83.175112,2015-07-18 07:26:51.975000064
min,12.990000,2015-07-15 07:32:00
25%,21.000000,2015-07-16 13:04:30
50%,89.000000,2015-07-17 16:28:00
75%,99.000000,2015-07-19 23:36:00
max,239.000000,2015-07-22 23:06:00
std,55.146881,NaN


### 3. Carga (Load)

In [ ]:
#definir ruta de salida para guardar el nuevo CSV
ruta_salida = '/content/drive/MyDrive/AnalisisDeDatos/ventas_modificadas.csv'
#guardar el dataframe transformado en un nuevo archivo CSV en Drive
df.to_csv(ruta_salida, index=False, sep = ',')
print(f'DataFrame guardado en: {ruta_salida}')

## 2 - EDA (Analisis Exploratorio de datos)

In [ ]:
# Cuanto facturo cada tienda.
print(df.groupby('tienda')['precio'].sum().reset_index())

#cuanto facturo por marca y tienda (ordenado)
facturacion_ordenada = df.groupby(['tienda','marca'])['precio'].sum().reset_index()
top = facturacion_ordenada.sort_values(by='precio', ascending=False)
print(top)
print(top.head(3))

       tienda    precio
0        Lima  49298.39
1  Provincias  17241.70
       tienda   marca    precio
1        Lima   Asics  24592.39
2        Lima    Nike  19633.00
4  Provincias   Asics  14119.70
0        Lima  Adidas   5073.00
5  Provincias    Nike   1938.00
3  Provincias  Adidas   1184.00


In [17]:
#Filtrado condicional -- Boolean Indexing

#df['categoria'].unique()
#df['color'].unique()
#1 Definir condicion: categoria = pantalon & Negro & genero
condicionF = (df['categoria'] == 'Pantalon') & (df['color'] == 'Negro') & (df['genero'] == 'F')
condicionM = (df['categoria'] == 'Pantalon') & (df['color'] == 'Negro') & (df['genero'] == 'M')
condicion = (df['categoria'] == 'Pantalon') & (df['color'] == 'Negro')

#2 Aplicar condicion
df_filtradoPantalonNegro = df[condicion]
df_filtradoPantalonNegroF = df[condicionF]
df_filtradoPantalonNegroM = df[condicionM]

#3 Preguntas
conteo_transacciones_PantalonNegro = len(df_filtradoPantalonNegro)
print(f'Conteo de pantalones negros vendidos: {conteo_transacciones_PantalonNegro}')

facturacion_total_PantalonNegro = df_filtradoPantalonNegro['precio'].sum()
print(f'Facturacion total: {facturacion_total_PantalonNegro}')

conteo_transacciones_PantalonNegroF = len(df_filtradoPantalonNegroF)
print(f'Conteo de pantalones negros vendidos Femeninos: {conteo_transacciones_PantalonNegroF}')

facturacion_total_PantalonNegroF = df_filtradoPantalonNegroF['precio'].sum()
print(f'Facturacion total Femenino: {facturacion_total_PantalonNegroF}')

conteo_transacciones_PantalonNegroM = len(df_filtradoPantalonNegroM)
print(f'Conteo de pantalones negros vendidos Masculino: {conteo_transacciones_PantalonNegroM}')

facturacion_total_PantalonNegroM = df_filtradoPantalonNegroM['precio'].sum()
print(f'Facturacion total Masculino: {facturacion_total_PantalonNegroM}')

#4 Respuesta: Se venden mas pantalones de color negro para genero Masculino


Conteo de pantalones negros vendidos: 165
Facturacion total: 16045.0
Conteo de pantalones negros vendidos Femeninos: 59
Facturacion total Femenino: 5551.0
Conteo de pantalones negros vendidos Masculino: 106
Facturacion total Masculino: 10494.0


In [24]:
#Extraccion de Componentes de Fecha y Hora

#1 Extraer el día de la semana como nombre
df['dia_semana'] = df['fecha_hora'].dt.day_name()

#2 Agrupar la facturacion por el nuevo dia de la semana
facturacion_por_dia = df.groupby('dia_semana')['precio'].sum().reset_index(name = 'total')

#3 ¿Que dia de la semana (lunes = 0 a Domingo 6 fue mas rentable en terminos de facturacion total?)
dia_mas_rentable = facturacion_por_dia.sort_values(by='total', ascending=False)
print('\n -- Analisis Temporal: Facturacion por Dia de la Semana --')
print(dia_mas_rentable)

#4 Respuesta : Se vende mas el dia Miercoles
# Consideraciones:
#4.1 Mas personal y reposicion de Stock el dia Miercoles.
#4.2 Reposicion y control de inventario dos dias antes (Lunes y Martes)



 -- Analisis Temporal: Facturacion por Dia de la Semana --
  dia_semana     total
6  Wednesday  16641.78
0     Friday  13447.90
4   Thursday  12363.86
2   Saturday   8029.84
5    Tuesday   6632.91
1     Monday   4835.91
3     Sunday   4587.89


In [26]:
#Pivot tables

# ¿Cual es el precio promedio de los productos, desglosado por tienda (en filas) y por categoria (en columnas)?

# Parámetro	Función
# index	La(s) columna(s) que se usarán como filas (eje X).
# columns	La(s) columna(s) que se usarán como columnas (eje Y).
# values	La columna sobre la cual se aplicará la agregación.
# aggfunc	La función de agregación a aplicar ('mean', 'sum', 'count', etc.).

#1 Metodo pivot table
tabla_promedio_tienda_categoria = pd.pivot_table (
    df,
    values = 'precio',
    index = 'tienda',
    columns = 'categoria',
    aggfunc = 'mean'          #Funcion de agregacion (promedio)

)

print('\n -- Analisis de Categoria por Tienda y Categoria --')
print(tabla_promedio_tienda_categoria)


 -- Analisis de Categoria por Tienda y Categoria --
categoria    Pantalon  Ropa interior  Sujetadores  Zapatillas
tienda                                                       
Lima        93.048780      19.942525   128.533333  166.513514
Provincias  93.206349      19.979365   122.230769  132.500000


In [27]:
# ¿Cual es la cantidad de ventas para cada genero, desglosado por tienda (filas) y categoria (columnas)?

#1 Metodo pivot table
tabla_cantidad_ventas_genero = pd.pivot_table (
    df,
    values = 'precio',
    index = 'genero',
    columns = 'categoria',
    aggfunc = 'count'          #Funcion de agregacion (conteo)
)

print('\n -- Analisis de Categoria por Genero --')
print(tabla_cantidad_ventas_genero)


 -- Analisis de Categoria por Genero --
categoria  Pantalon  Ropa interior  Sujetadores  Zapatillas
genero                                                     
F             225.0           40.0         73.0        37.0
M             106.0            NaN          NaN        94.0
U               NaN          225.0          NaN         NaN


In [28]:
# ¿Cual es la cantidad de ventas de todos los productos, desglosado por tienda (filas) y categoria (columnas)?

#1 Metodo pivot table
tabla_cantidad_ventas_tienda_categoria = pd.pivot_table (
    df,
    values = 'precio',
    index = 'tienda',
    columns = 'categoria',
    aggfunc = 'count'          #Funcion de agregacion (conteo)
)

print('\n -- Analisis de Categoria por Tienda --')
print(tabla_cantidad_ventas_tienda_categoria)


 -- Analisis de Categoria por Tienda --
categoria   Pantalon  Ropa interior  Sujetadores  Zapatillas
tienda                                                      
Lima             205            202           60         111
Provincias       126             63           13          20
